In [1]:
import time
from running_dinner_ga.ga_algorithm import run_ga
from running_dinner_ga.Individual_RD import IndividualRD
import pandas as pd

In [2]:
import time
from running_dinner_ga.ga_algorithm import run_ga
from running_dinner_ga.Individual_RD import IndividualRD
import pandas as pd

coords_df = pd.read_csv('Data/coordinates_map.csv', index_col='Index')
house_coords = coords_df[['Latitude','Longitude']].values[:10]

# Create one individual and randomize it
ind = IndividualRD(house_coords, n_participants=18, capacity_of_houses=6, a=1.0)
ind.random_representation()

# Time its fitness
t0 = time.perf_counter()
fit = ind.fitness
t1 = time.perf_counter()
print(f"Single-ind fitness: {fit:.3f} (computed in {t1-t0:.4f}s)")

Single-ind fitness: 2.372 (computed in 0.0175s)


In [3]:
import time, pandas as pd
from running_dinner_ga.ga_algorithm import run_ga

coords_df    = pd.read_csv('Data/coordinates_map.csv', index_col='Index')
house_coords = coords_df[['Latitude','Longitude']].values[:10]

t0 = time.perf_counter()
_  = run_ga(
    pop_size=20,
    house_coords=house_coords,
    n_participants=18,
    capacity=6,
    a=1.0,
    generations=1,
    selection_method='tournament',
    tournament_size=3
)
print("Total elapsed:", time.perf_counter() - t0)


>>> Generation 1/1 start
   loop 10: new_pop size = 20
<<< Generation 1 done in 0.487s (loops: 10)

GA complete.
Total elapsed: 0.5080469070235267


In [1]:
# === RUN 10 GENERATIONS & SHOW BEST GENOME ===

from running_dinner_ga.ga_algorithm import run_ga
import pandas as pd

# 1) Reload coordinates (first 10 as houses)
coords_df    = pd.read_csv('Data/coordinates_map.csv', index_col='Index')
house_coords = coords_df[['Latitude','Longitude']].values[:10]

# 2) GA parameters
n_participants = 18
capacity       = 6
a               = 1.0
pop_size        = 20
generations     = 10
tournament_k    = 3

# 3) Run the GA for 10 gens
final_population = run_ga(
    pop_size=pop_size,
    house_coords=house_coords,
    n_participants=n_participants,
    capacity=capacity,
    a=a,
    generations=generations,
    selection_method='tournament',
    tournament_size=tournament_k
)

# 4) Find and print the best solution’s genome
best = min(final_population, key=lambda ind: ind.fitness)
print("Best fitness:", best.fitness)
print("Best genome:", best.solution.genome)


>>> Generation 1/10 start
   loop 10: new_pop size = 20
<<< Generation 1 done in 0.539s (loops: 10)

>>> Generation 2/10 start
   loop 10: new_pop size = 20
<<< Generation 2 done in 0.454s (loops: 10)

>>> Generation 3/10 start
   loop 10: new_pop size = 20
<<< Generation 3 done in 0.328s (loops: 10)

>>> Generation 4/10 start
   loop 10: new_pop size = 20
<<< Generation 4 done in 0.564s (loops: 10)

>>> Generation 5/10 start
   loop 10: new_pop size = 20
<<< Generation 5 done in 0.531s (loops: 10)

>>> Generation 6/10 start
   loop 10: new_pop size = 20
<<< Generation 6 done in 0.137s (loops: 10)

>>> Generation 7/10 start
   loop 10: new_pop size = 20
<<< Generation 7 done in 0.167s (loops: 10)

>>> Generation 8/10 start
   loop 10: new_pop size = 20
<<< Generation 8 done in 0.170s (loops: 10)

>>> Generation 9/10 start
   loop 10: new_pop size = 20
<<< Generation 9 done in 0.192s (loops: 10)

>>> Generation 10/10 start
   loop 10: new_pop size = 20
<<< Generation 10 done in 0.449s 